In [1]:
#Imports
from pyspark import SparkContext #Library
sc = SparkContext("local[*]", "temp") #Start a spart context

from pyspark.ml.feature import Word2Vec
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

import xml.etree.ElementTree as ET



In [16]:
posts_lines = sc.textFile("file:///home/vagrant/miniprojects/spark/full/allPosts/*.gz") #create an RDD from file
print posts_lines
posts_lines.count()


MapPartitionsRDD[86] at textFile at NativeMethodAccessorImpl.java:-2


47946654

In [17]:
def parsePost(line):
    if '<row'in line:
        try:
            root = ET.fromstring(line.encode('utf8').strip())
        except:
            pass
            return ("Empty")
            
        if root != '':
            if "Tags" in root.attrib:
                    try:
                        data = root.attrib["Tags"]
                    except:
                        pass
                        return ("Empty")
                    data = data.replace("<", " ").replace(">", " ")
                    return data
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")
            
        

In [18]:

content = posts_lines.map(parsePost).filter(lambda x: x!= 'Empty')\
            .map(lambda line: (line.split(" "), 1)).toDF(['text', 'score'])

print content
content.count()
print content.take(20)

DataFrame[text: array<string>, score: bigint]
[Row(text=[u'', u'optimization', u'', u'cuda', u''], score=1), Row(text=[u'', u'java', u'', u'android', u'', u'linux', u'', u'networking', u'', u'sshj', u''], score=1), Row(text=[u'', u'python', u'', u'python-3.x', u'', u'dictionary', u''], score=1), Row(text=[u'', u'android', u''], score=1), Row(text=[u'', u'solr', u'', u'elasticsearch', u'', u'morelikethis', u''], score=1), Row(text=[u'', u'android', u'', u'android-intent', u'', u'call', u''], score=1), Row(text=[u'', u'ruby', u'', u'dictionary', u''], score=1), Row(text=[u'', u'java', u'', u'android', u'', u'class', u'', u'bluetooth', u'', u'android-bluetooth', u''], score=1), Row(text=[u'', u'jsp', u'', u'error-handling', u'', u'struts-1.x', u''], score=1), Row(text=[u'', u'bash', u'', u'arguments', u''], score=1), Row(text=[u'', u'codeigniter', u''], score=1), Row(text=[u'', u'c++', u'', u'class', u'', u'templates', u''], score=1), Row(text=[u'', u'c++', u'', u'cmake', u''], score=1), 

In [19]:
w2v = Word2Vec(inputCol="text",vectorSize=100, seed=42L) 
model = w2v.fit(content)


In [20]:
result = model.findSynonyms('ggplot2', 40).rdd.take(40)
print result


[Row(word=u'lattice', similarity=3.524193114110293), Row(word=u'data.table', similarity=3.3310135983354088), Row(word=u'plyr', similarity=3.3025323727585065), Row(word=u'data.frame', similarity=3.289413454401584), Row(word=u'boxplot', similarity=3.283173644508127), Row(word=u'shiny', similarity=3.229499163582802), Row(word=u'dplyr', similarity=3.1751775821156856), Row(word=u'reshape', similarity=3.137661829586672), Row(word=u'xts', similarity=3.135725747540133), Row(word=u'plot', similarity=3.0837451419287265), Row(word=u'rstudio', similarity=3.0748395155952974), Row(word=u'time-series', similarity=3.0575445126499283), Row(word=u'legend', similarity=3.0381227646791773), Row(word=u'lm', similarity=3.035470365285809), Row(word=u'ggmap', similarity=3.011285446212616), Row(word=u'reshape2', similarity=3.0104954805609343), Row(word=u'axis-labels', similarity=2.994953293973785), Row(word=u'quantmod', similarity=2.993072617965781), Row(word=u'rgl', similarity=2.9858464230342263), Row(word=u'f

In [21]:

answer=[]
for i in range(0,25):
    answer.append((str(result[i][0]), result[i][1]))
print answer

[('lattice', 3.524193114110293), ('data.table', 3.3310135983354088), ('plyr', 3.3025323727585065), ('data.frame', 3.289413454401584), ('boxplot', 3.283173644508127), ('shiny', 3.229499163582802), ('dplyr', 3.1751775821156856), ('reshape', 3.137661829586672), ('xts', 3.135725747540133), ('plot', 3.0837451419287265), ('rstudio', 3.0748395155952974), ('time-series', 3.0575445126499283), ('legend', 3.0381227646791773), ('lm', 3.035470365285809), ('ggmap', 3.011285446212616), ('reshape2', 3.0104954805609343), ('axis-labels', 2.994953293973785), ('quantmod', 2.993072617965781), ('rgl', 2.9858464230342263), ('facet-wrap', 2.966235297053326), ('geom-bar', 2.9635844933024518), ('heatmap', 2.958948121687268), ('r-factor', 2.952065058289612), ('regression', 2.946026635030315), ('zoo', 2.9265415841897378)]
